# Preamble

The goal of this is as a first, very basic, proof of concept of the idea to use a neural network to break basic codes. The first code we attempt to break is a Caesar cipher. To do so we use a CNN to classify the encrypted text by the key that will decrypt it. This approach works well, when trained on the Guttenberg Project from [Hugging Face](https://huggingface.co/datasets/manu/project_gutenberg) we get very close to 100% accuracy with small amounts of training time and no hyperparameter tuning. The structure of the model was largely inspired by [Understanding Convolutional Neural Networks for NLP](https://dennybritz.com/posts/wildml/understanding-convolutional-neural-networks-for-nlp/).

## Variables

In [ ]:
DATA_AMOUNT = [10000, 1000]
TRAIN_SPLIT = 0.8
STREAM = False

In [ ]:
BATCH_SIZE = 64
EMBEDDING_DIM = 128
NUM_FILTERS = 128
FILTER_SIZES = [2, 3, 4, 5, 7]
DROPOUT_PROB = 0.5
NUM_CLASSES = 25
VOCAB_SIZE = 28
NUM_EPOCHS = 10
LR = 0.01

## Imports

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from modules import caesar_cnn_data

from tqdm.autonotebook import tqdm

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

## Creating Data

In [ ]:
trainData, testData = caesar_cnn_data.initialise(*DATA_AMOUNT, TRAIN_SPLIT, stream=STREAM)
train_loader, test_loader = caesar_cnn_data.data2loader(trainData, testData, BATCH_SIZE=BATCH_SIZE)

# CNN Model

In [ ]:
class CNNCodeBreaker(nn.Module):
    def __init__(self,
                 num_filters=NUM_FILTERS,
                 filter_sizes=FILTER_SIZES,
                 num_classes=NUM_CLASSES,
                 dropout_prob=DROPOUT_PROB,
                 embed_dim=EMBEDDING_DIM,
                 vocab_size=VOCAB_SIZE):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=27)
        self.convs = nn.ModuleList([
            nn.Conv1d(in_channels=embed_dim, out_channels=num_filters, kernel_size=k)
            for k in filter_sizes
        ])
        self.linear = nn.Linear(len(filter_sizes) * num_filters, num_classes)
        self.dropout = nn.Dropout(dropout_prob)

    def forward(self, x):
        embedded = self.embedding(x).permute(0, 2, 1)
        conved = [F.relu(conv(embedded)) for conv in self.convs]
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        cat = torch.cat(pooled, dim=1)
        dropped = self.dropout(cat)
        return self.linear(dropped)

In [ ]:
model = CNNCodeBreaker().to(device)
model

# Training

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimiser = optim.Adam(model.parameters(), lr=LR)

In [ ]:
for epoch in tqdm(range(NUM_EPOCHS)):
    model.train()
    running_loss = 0

    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimiser.zero_grad()
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimiser.step()
        running_loss += loss.item()

    avg_train_loss = running_loss / len(train_loader)

    model.eval()
    test_loss = 0
    correct = 0
    total = 0

    with torch.inference_mode():
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            loss = loss_fn(outputs, labels)
            test_loss += loss.item()

            predicted = torch.argmax(outputs, dim=1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_test_loss = test_loss / len(test_loader)
    accuracy = 100 * correct / total
    print(f"Epoch {epoch:02d}/{NUM_EPOCHS} | Train Loss: {avg_test_loss:.4f} | Test Loss: {avg_test_loss:.4f} | Test Accuracy: {accuracy:.1f}%")

## Saving

In [ ]:
torch.save(model.state_dict(), "models/00_caesar_cnn.pt")

# Example Useage

In [ ]:
model.eval()
test_string = "The quick brown fox jumps over the lazy dog"
true_key = 14
enc_text, _ = caesar.encrypt(test_string, key=true_key)
print(f"Original Text: '{test_string}'")
print(f"Encrypted Text: '{enc_text}'")
input_tensor = torch.tensor(get_text.string2_num_list(enc_text), dtype=torch.long).unsqueeze(0).to(device)
with torch.inference_mode():
    pred_logits = model(input_tensor)
pred_key = torch.argmax(pred_logits, dim=1) + 1
print(f"Model Prediction (Key): {pred_key.item}")